In [1]:
import numpy as np
import scipy as sp
from scipy import fftpack

import pandas as pd

import os
import glob
from sklearn.preprocessing import StandardScaler


In [2]:
path = os.getcwd() + '\\data\\'
print(os.getcwd())
extension = 'csv'

os.chdir(path)
titles = glob.glob('*.{}'.format(extension))

C:\Users\tlpel\Documents\Studie\AI1\pAI\code


In [3]:
datasetlist = []
    
for d in titles:
    data = pd.read_csv(path + d)
    if len(data.columns) == 1:
        data = pd.read_csv(path + d, sep=";")
    data['action'] = data['Stimulus'].apply(lambda x: x.replace(' ', '.').split('.')[0])
    # last row is buggy
    data = data[:-1]
    datasetlist.append(data[data.columns[1:]])

In [4]:
# 

In [24]:
# put all pandas frames together
datasettrain = datasetlist[:-1]
datasettest = datasetlist[-1]
datasets = pd.concat(datasettrain)
print(len(datasets))


22605


In [25]:
def extract_action(stimulus):
    action_nr = stimulus.split('.')[0]
    splitted = action_nr.split('-')
    action = splitted[0]
    if len(splitted) == 1:
        nr = '0'
    else:
        nr = splitted[1]
    return action.strip(), nr.strip()

In [26]:
actions = {}

data = datasets

data = data['action']
i = 0

for d in data:
    action,number = extract_action(d)

    if not action in list(actions.keys()):
        actions[action] = i
        i += 1
        print('added', action)
        
action_length = len(actions)

added crouch
added walk
added run
added jump


In [45]:
from sklearn.decomposition import PCA

## Split the signals into small subsets with an overlap
def subset_data(data, ss_dim = None, ss_num = 10, overlap = .5, cut_smaller = True):
    if overlap > 1: # in case the value passed is a percentage
        overlap = float(overlap)/100
        
    if ss_dim is None: # either choose the dimension of the subsets or the number
        ss_dim = int(len(data)/(ss_num*overlap)) # by default it will divide the signal in 10 subsets
    
    subsets = []
    i = len(data) - 1
    while i >= 0:
        j = max(i - ss_dim, 0)
        subsets.append(data[j:i])
        i -= int(ss_dim * (1 - overlap))
    
    while cut_smaller and len(subsets[-1]) < ss_dim :
        subsets = subsets[:-1]
        
    return np.array(subsets)

def prepare_data(dataframe, ss_dim = 20, ss_num = 168, overlap = .5):
    # PCA decorrelation
    X = dataframe.ix[:,1:15].values
    pca = PCA(n_components=14)
    Z = pca.fit_transform(np.array(X))
    for i in range(Z.shape[1]):
        dataframe.ix[:,i+1] = Z[:,i]
    print(dataframe)
    
    # split on actions
    dataset = {}
    for action in set(dataframe.action.values):
        a = []
        data = dataframe[dataframe.action == action].copy()
        for c in data.columns[1:-1]:
            a.append(np.array(subset_data(np.array(data[c]), ss_dim, ss_num, overlap, cut_smaller = True)))
        dataset[action] = [np.asmatrix(d).flatten() for d in np.transpose(np.array(a), [1, 2, 0])] # if you want a list of matrixes
#         dataset[action] = np.transpose(np.array(a), [1, 2, 0]) # if you want a tensor
    return dataset

In [46]:
# preprocess the data
# performs normalization on the data and splits it out in actions, possibly batches the action signals into smaller subsignals
# pca
dataset = prepare_data(datasets, ss_dim = 20, overlap = 0.5)
datatest = prepare_data(datasettest, ss_dim = 20, overlap = 0.5)
print(len(dataset["walk"]))



                    Stimulus         AF3         F7         F3        FC5  \
0      crouch - Copy (2).png -527.783395  -0.216958  77.500615  50.884510   
1      crouch - Copy (2).png -504.309141   3.611238  73.044664  34.070391   
2      crouch - Copy (2).png -523.615749   3.789307  69.551235  31.823897   
3      crouch - Copy (2).png -534.616412   5.035321  69.397469  31.502522   
4      crouch - Copy (2).png -537.118549   5.347372  72.625622  24.972982   
5      crouch - Copy (2).png -554.197072  -9.556444  74.352628  21.462045   
6      crouch - Copy (2).png -555.195207 -24.542502  69.620452  19.071141   
7      crouch - Copy (2).png -536.083697 -15.505064  68.948296  17.144859   
8      crouch - Copy (2).png -532.072725  -1.903233  77.512702  19.730814   
9      crouch - Copy (2).png -540.676091  -0.071301  82.488817  21.248181   
10     crouch - Copy (2).png -527.733936  -2.097342  79.036634  20.646466   
11     crouch - Copy (2).png -506.494983  -3.790514  73.220664  22.380703   

In [47]:

# put the data and labels in two seperate lists
def get_XY(dataset):
    X = None
    Y = None
    for t in dataset.keys():
        x = np.squeeze(dataset[t], axis = 1)

        if X is None:
            X = x   
        else:
            X = np.concatenate( (X, x))


        y = np.array([actions[t]])
        y = np.tile(y,len(x))

        if Y is None:
            Y = y
        else:
            Y = np.concatenate((Y,y))
            
    # scale the data to mean 0 and variance 1
    X = StandardScaler().fit_transform(X)
    
    return X, Y

X, Y = get_XY(dataset)
print(X.shape)
Xt, Yt = get_XY(datatest)
print(Xt.shape)
print(Yt.shape)

(2254, 280)
(1127, 280)
(1127,)


In [48]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from random import shuffle

# cutoff = round(len(X) * 0.5)  #80-20 division

# # shuffle data
# Z = list(zip(X,Y))
# shuffle(Z)

# X,Y = zip(*Z)
# X = list(X)
# Y = list(Y)

# # extract training and test set
# X_train = np.array(X[0:cutoff])
# y_train = np.array(Y[0:cutoff])
# X_test = np.array(X[cutoff: -1])
# y_test = np.array(Y[cutoff: -1])

# inputs
inp = X
out = Y

def classify_net(X, Y, Xt, Yt):
    clf = MLPClassifier(solver="lbfgs", alpha=1e-4,hidden_layer_sizes=(100, 50, 25),random_state=1, max_iter=2000)
    print(clf.fit(X, Y))
    print(clf.predict(Xt))
    print(clf.score(Xt,Yt))
    
def classify_knn(X, Y, Xt, Yt, n):
    clf = KNeighborsClassifier(n_neighbors=n, weights='distance')
    print(clf.fit(X, Y))
    #print(clf.predict(inp))
    print(clf.score(Xt,Yt))
    

# run classifiers
classify_net(inp,out,Xt,Yt)
for i in range(1,10):
    print(i)
    classify_knn(inp,out,Xt,Yt,i)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 50, 25), learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)
[1 1 1 ..., 3 0 3]
0.250221827862
1
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='distance')
0.241348713398
2
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')
0.241348713398
3
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='distan